In [22]:
from numpy.random import choice, seed
import numpy as np
from scipy.stats import norm
import matplotlib.pyplot as plt
from numpy import sqrt
import pandas as pd
import seaborn as sns

## Simulating the distribution of a brownian motion at $t=1$ (S 3.2)

In [2]:
symmetric_random_walk = lambda nsteps: np.cumsum(np.append(0, choice([-1,1], nsteps)))
scaled_random_walk  = lambda partitions, t: symmetric_random_walk(partitions * t) / sqrt(partitions)

In [34]:
#seed(1994)
samples = 500
partitions = [10, 20, 50, 100, 500]

vals = {}
for partition in partitions:
    # Applying LLN
    means = []
    for __ in range(samples):
        samp_values = []
        # Running the si
        for _ in range(samples):
            last_pos = scaled_random_walk(partition,1)[-1]
            # Getting values that fall between two standard deviations
            samp_values.append(last_pos)
        means.append(np.mean([abs(p) <= 2 for p in samp_values]))
            
    vals[str(partition)] = np.mean(means)

In [48]:
vals

{'10': 0.97846399999999989,
 '100': 0.96461600000000003,
 '20': 0.95863199999999993,
 '50': 0.96706399999999992,
 '500': 0.95598800000000006}